MIS 285N Cognitive Computing<br>
Final Project<br>
Jerry Che - Jose Guerrero - Riley Moynihan - Noah Placke - Sarah Teng - Palmer Wenzel

# Instructions Generation Model

Following techniques from:
- https://towardsdatascience.com/generative-adversarial-networks-in-python-73d3972823d3
- https://www.maskaravivek.com/post/gan-synthetic-data-generation/

#### Read data from CSV.

In [1]:
import pandas as pd
# pd.options.display.max_columns = 500


df = pd.read_csv('../data/kaggle/processed/recipes_processed.csv')#.sample(frac=0.1, random_state=42)

df.head(3)

,name,steps,crabmeat,creamcheese,greenonions,garlicsalt,refrigeratedcrescentdinnerrolls,eggyolk,water,sesameseeds,...,tex-mexseasoning,lightnon-dairywhippedtopping,stelladoroanginetticookies,viennabread,beefroundrumproast,romaineleaf,nuocnam,thaiholybasil,driedblacktrumpetmushrooms,driedwoodearmushrooms
0,crab filled crescent snacks,"heat over to 375 degrees, spray large cookie s...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,curried bean salad,"drain & rinse beans, stir all ingredients toge...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,delicious steak with onion marinade,heat the oil in a heavy-based pan and cook the...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Drop unnecessary columns.

In [2]:
df = df.drop(['name'], axis=1)

df.head(3)

,steps,crabmeat,creamcheese,greenonions,garlicsalt,refrigeratedcrescentdinnerrolls,eggyolk,water,sesameseeds,sweetandsoursauce,...,tex-mexseasoning,lightnon-dairywhippedtopping,stelladoroanginetticookies,viennabread,beefroundrumproast,romaineleaf,nuocnam,thaiholybasil,driedblacktrumpetmushrooms,driedwoodearmushrooms
0,"heat over to 375 degrees, spray large cookie s...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"drain & rinse beans, stir all ingredients toge...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,heat the oil in a heavy-based pan and cook the...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Separate steps from ingredients.

In [5]:
steps = df['steps']
ingredients = df.drop(['steps'], axis=1)

#### Create vocabulary of unique words.

In [6]:
unique = set(steps.str.replace('[^a-zA-Z0-9 ]', '').str.split(' ').sum())
print(f'Number of unique tokens: {len(unique)}')

Number of unique tokens: 16902


#### Create index to word mappings.

In [10]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in unique:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1
    
ixtoword[3]

'forks'

#### Load InceptionV3

In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3


# Load in pretrained model
inception_model = InceptionV3(include_top=False, input_shape=(ingredients.shape[1],))

# Remove output layer so we can retrain it
model = Model(inception_model.input, inception_model.layers[-2].output)

In [ ]:
for 